In [10]:
import pandas as pd
import numpy as np

In [11]:
# categorical_df = pd.read_csv('../dataset/5_categorical_data.csv')
# numerical_df = pd.read_csv('../dataset/5_numerical_data.csv')

categorical_df = pd.read_csv('../dataset/categorical_data.csv')
numerical_df = pd.read_csv('../dataset/numerical_data.csv')


categorical_df.info()

/var/folders/xk/fhgbj8qj60n54b05mpfnck0w0000gn/T/ipykernel_15791/2579389052.py:4: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  categorical_df = pd.read_csv('../dataset/categorical_data.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155361 entries, 0 to 155360
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   symmetry       155361 non-null  object
 1   structure      155361 non-null  object
 2   decomposes_to  118418 non-null  object
 3   xas            60920 non-null   object
 4   is_gap_direct  155353 non-null  object
 5   is_metal       155353 non-null  object
 6   bandstructure  71727 non-null   object
 7   dos            89547 non-null   object
 8   ordering       154379 non-null  object
 9   bulk_modulus   12259 non-null   object
 10  shear_modulus  12259 non-null   object
dtypes: object(11)
memory usage: 13.0+ MB


In [12]:
#begin work on symmetry attribute
import ast

crystal_sytems_df = []

for row in categorical_df.itertuples():
   
    symmetry_dict = ast.literal_eval(row.symmetry) 
    crystal_sytems_df.append(symmetry_dict['crystal_system'])

crystal_sytems_df = pd.get_dummies(
    crystal_sytems_df, columns=['crystal_system'], 
                                    drop_first=True, prefix='crystal_sys')

crystal_sytems_df = crystal_sytems_df.astype(int)
categorical_df = categorical_df.drop('symmetry', axis=1)

numerical_df = pd.concat([numerical_df, crystal_sytems_df], axis=1)
numerical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155361 entries, 0 to 155360
Data columns (total 43 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   nsites                                        155361 non-null  int64  
 1   nelements                                     155361 non-null  int64  
 2   volume                                        155361 non-null  float64
 3   density                                       155361 non-null  float64
 4   density_atomic                                155361 non-null  float64
 5   deprecated                                    155361 non-null  int64  
 6   energy_per_atom                               152149 non-null  float64
 7   formation_energy_per_atom                     152149 non-null  float64
 8   energy_above_hull                             152149 non-null  float64
 9   is_stable                                     15

In [13]:
import ast

lattice_matrix_df = []

for row in categorical_df.itertuples(index=False):
    try:
        structure_dict = ast.literal_eval(row.structure)
        
        if 'lattice' in structure_dict and 'matrix' in structure_dict['lattice']:
            lattice_matrix = structure_dict['lattice']['matrix']
            lattice_matrix_df.append(lattice_matrix)

    except (ValueError, SyntaxError) as e:
        lattice_matrix_df.append("none")
lattice_matrix_df = pd.DataFrame(lattice_matrix_df)

print(lattice_matrix_df.head())

                                       0  \
0                   [4.934894, 0.0, 0.0]   
1   [12.49654657, 0.17686483, 0.2294622]   
2       [10.247851, -0.106168, 0.008782]   
3  [11.8521467, 0.55559273, -0.00317713]   
4  [11.61272298, 0.89401036, 1.03378931]   

                                        1  \
0              [-2.219485, 4.913732, 0.0]   
1  [0.17823107, 11.45138636, -0.31319981]   
2        [-0.098855, 10.894242, 0.432224]   
3   [0.55644679, 11.72989848, 0.31759797]   
4   [0.79773741, 13.36727246, -0.8604892]   

                                        2  
0         [-2.418904, -1.422197, 6.56246]  
1  [0.22747837, -0.31303906, 12.33339993]  
2         [0.005544, 0.456147, 10.113102]  
3  [-0.01663057, 0.31487646, 11.71135177]  
4  [1.03530586, -0.94340276, 12.59653651]  


In [14]:
#Convert matrices to a single value using Frobeius norm:

def frobenius_norm(matrix_str):
    try:
        matrix = np.array(ast.literal_eval(matrix_str))
        return np.linalg.norm(matrix, 'fro')
    except (ValueError, SyntaxError):
        return None
    
norms_df = []

for row in lattice_matrix_df.itertuples(index=False):
    matrix=row[0]
    norm = frobenius_norm(matrix)
    norms_df.append(norm)

norms_df = pd.DataFrame(norms_df)
print(norms_df)

           0
0       None
1       None
2       None
3       None
4       None
...      ...
155356  None
155357  None
155358  None
155359  None
155360  None

[155361 rows x 1 columns]


In [15]:
numerical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155361 entries, 0 to 155360
Data columns (total 43 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   nsites                                        155361 non-null  int64  
 1   nelements                                     155361 non-null  int64  
 2   volume                                        155361 non-null  float64
 3   density                                       155361 non-null  float64
 4   density_atomic                                155361 non-null  float64
 5   deprecated                                    155361 non-null  int64  
 6   energy_per_atom                               152149 non-null  float64
 7   formation_energy_per_atom                     152149 non-null  float64
 8   energy_above_hull                             152149 non-null  float64
 9   is_stable                                     15

In [16]:
categorical_df['is_metal'] = categorical_df['is_metal'].fillna(0).astype(int)
categorical_df['is_gap_direct'] = categorical_df['is_gap_direct'].fillna(0).astype(int)

numerical_df = pd.concat([numerical_df, categorical_df[['is_metal', 'is_gap_direct']]], axis=1)

categorical_df = categorical_df.drop(columns=['is_metal', 'is_gap_direct'])

categorical_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155361 entries, 0 to 155360
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   structure      155361 non-null  object
 1   decomposes_to  118418 non-null  object
 2   xas            60920 non-null   object
 3   bandstructure  71727 non-null   object
 4   dos            89547 non-null   object
 5   ordering       154379 non-null  object
 6   bulk_modulus   12259 non-null   object
 7   shear_modulus  12259 non-null   object
dtypes: object(8)
memory usage: 9.5+ MB


In [17]:
numerical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155361 entries, 0 to 155360
Data columns (total 45 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   nsites                                        155361 non-null  int64  
 1   nelements                                     155361 non-null  int64  
 2   volume                                        155361 non-null  float64
 3   density                                       155361 non-null  float64
 4   density_atomic                                155361 non-null  float64
 5   deprecated                                    155361 non-null  int64  
 6   energy_per_atom                               152149 non-null  float64
 7   formation_energy_per_atom                     152149 non-null  float64
 8   energy_above_hull                             152149 non-null  float64
 9   is_stable                                     15

In [18]:
numerical_df.to_csv('../dataset/cleaned_numerical_data.csv', index=False)
categorical_df.to_csv('../dataset/cleaned_categorical_data.csv', index=False)
